In [6]:
import cv2
import numpy as np
import psutil
import os
from tqdm import tqdm

p = psutil.Process(os.getpid())
p.cpu_affinity([0, 1, 2, 3, 4, 5, 6, 7])

# 要讀取的.MTS文件名
filename = (r"G:\works\Data_archive\8f_behav\grid_walking\2023_07_21\LMO_G226_CTZ_gridwalk_0721.MP4")

#output
width, height = 480, 480
fps = 100
savefilename = ["LMO_G226_CTZ_gridwalk_0721_bottomview.MP4"]
savedir = (r"G:\works\Data_archive\8f_behav\grid_walking\2023_07_21\crop\bottom_view\\")
ROI_num = 1
start_time = [1] #sec
duration = 5 #min
ispreview = False


# 創建VideoCapture對象
cap = cv2.VideoCapture(filename)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# 檢查文件是否成功打開
if not cap.isOpened():
    print('無法打開文件')

# 建立 VideoWriter 物件
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# 創建窗口並設置鼠標回調函數
cv2.namedWindow('frame')
points = []
Ms = []

def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        if len(points) == ROI_num*4:
            cv2.setMouseCallback('frame', lambda *args: None)

cv2.setMouseCallback('frame', mouse_callback)

previewed = False

# 開始讀取視頻幀
while True:
    ret, frame = cap.read()
    
    # 當讀取完所有幀，退出循環
    if not ret:
        break
        
    resized_frame = cv2.resize(frame, (1600, 900))
        
    # 在每次點擊時繪製圓圈
    for point in points:
        cv2.circle(resized_frame, point, 5, (0, 0, 255), -1)
        
    # 等待用戶選擇四個點
    if len(points) < ROI_num*4:
        cv2.imshow('frame', resized_frame)
        cv2.waitKey(1)
        continue
        
    # 計算變換矩陣
    dst_points = np.float32([[0, 0], [height, 0], [height, width], [0, width]])
    
    for i in range(ROI_num):
        src_points = np.float32((points[0+i*4],points[1+i*4],points[2+i*4],points[3+i*4]))
        M = cv2.getPerspectiveTransform(src_points, dst_points)
        Ms.append(M)
    break
    
for i in range(ROI_num):
    outputfull = savedir+savefilename[i]
    out = cv2.VideoWriter(outputfull, fourcc, fps, (width, height))
    cap = cv2.VideoCapture(filename)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_time[i]*12.5)
    end_frame = (60*duration)*fps
    now_frames = start_time[i]*fps/2
    
    for now_frames in tqdm(range(end_frame)):  # 使用tqdm迭代
        ret, frame = cap.read()
        
        # 當讀取完所有幀，退出循環
        if not ret or now_frames >= end_frame :
            break
            
        if now_frames >= end_frame :    
            print('break')
            
        # 應用變換矩陣
        resized_frame = cv2.resize(frame, (1600, 900))
        warped = cv2.warpPerspective(resized_frame, Ms[i], (height, width))
        resized_frame = cv2.resize(warped, (480, 480))
        
        # 在窗口中顯示幀
        if ispreview:
            cv2.imshow('MTS Viewer', resized_frame)

            # 等待按鍵
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyWindow('MTS Viewer')
                break

            continue

        if not ispreview:
            out.write(resized_frame)
    
# 釋放資源和關閉視窗    
cap.release()
out.release()    
cv2.destroyAllWindows()

100%|██████████| 30000/30000 [03:10<00:00, 157.23it/s]
